In [7]:
import optuna
from optuna.trial import TrialState

In [5]:
def objective(trial):
    x = trial.suggest_float('x', -10, 10)
    return (x - 2) ** 2

study = optuna.create_study()
study.optimize(objective, n_trials = 100)

study.best_params

[I 2026-01-29 10:12:02,131] A new study created in memory with name: no-name-caeec899-7dfe-4997-97e3-dec27f482aae


[I 2026-01-29 10:12:02,184] Trial 0 finished with value: 0.17533503684381302 and parameters: {'x': 2.418730267408284}. Best is trial 0 with value: 0.17533503684381302.
[I 2026-01-29 10:12:02,186] Trial 1 finished with value: 35.771761269064434 and parameters: {'x': -3.9809498634468117}. Best is trial 0 with value: 0.17533503684381302.
[I 2026-01-29 10:12:02,187] Trial 2 finished with value: 86.72582220732045 and parameters: {'x': -7.312669982734299}. Best is trial 0 with value: 0.17533503684381302.
[I 2026-01-29 10:12:02,188] Trial 3 finished with value: 4.507561744891354 and parameters: {'x': 4.123101915804174}. Best is trial 0 with value: 0.17533503684381302.
[I 2026-01-29 10:12:02,189] Trial 4 finished with value: 22.759933015279245 and parameters: {'x': -2.7707371563815215}. Best is trial 0 with value: 0.17533503684381302.
[I 2026-01-29 10:12:02,190] Trial 5 finished with value: 20.47795807825745 and parameters: {'x': -2.525257791359234}. Best is trial 0 with value: 0.1753350368438

{'x': 2.032180650566154}

In [8]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms

In [10]:
DEVICE = torch.device('cpu')
BATCHSIZE = 128
CLASSES = 10
EPOCHS = 10
N_TRAIN_EXAMPLES = BATCHSIZE * 30
N_VALID_EXAMPLES = BATCHSIZE * 10

In [17]:
def define_model(trial):
    n_layers = trial.suggest_int('n_layers', 1, 3)
    layers = []

    in_features = 28 * 28
    for i in range(n_layers):
        out_features = trial.suggest_int('n_units_l{}'.format(i), 4, 128)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float('dropout_l{}'.format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, CLASSES))
    layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)

In [23]:
def get_mnist():

    train_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST(
            root = 'data',
            train = True,
            download = True,
            transform = transforms.ToTensor()
        ),
        batch_size = BATCHSIZE,
        shuffle = True
    )

    valid_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST(
            root = 'data',
            train = False,
            download = True,
            transform = transforms.ToTensor()
        ),
        batch_size = BATCHSIZE,
        shuffle = True
    )

    return train_loader, valid_loader

In [ ]:
def objective(trial):

    model = define_model(trial).to(DEVICE)

    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'RMSprop', 'SGD'])
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log = True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr = lr)
    train_loader, valid_loader = get_mnist()
    for epoch in range(EPOCHS):

        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            if batch_idx * BATCHSIZE >= N_TRAIN_EXAMPLES:
                break

            data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)

            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()

        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(valid_loader):
                if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
                    break
            
                data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
                output = model(data)

                pred = output.argmax(dim = 1, keepdim = True)
                correct += pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / min(len(valid_loader.dataset), N_VALID_EXAMPLES)

        trial.report(accuracy, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    
    return accuracy

In [29]:
study = optuna.create_study(direction = 'maximize')
study.optimize(objective, n_trials = 100, timeout = 600)

pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print('Study statistics: ')
print('  Number of finished trials: ', len(study.trials))
print('  Number of pruned trials: ', len(pruned_trials))
print('  Number of completed trials: ', len(complete_trials))

print('Best trials:')
trial = study.best_trial

print('  Value: ', trial.value)
print('  Params: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

[I 2026-01-29 11:20:35,954] A new study created in memory with name: no-name-1963ccf5-03c7-41df-8bf6-7daae3463a4c
[I 2026-01-29 11:20:40,306] Trial 0 finished with value: 0.1515625 and parameters: {'n_layers': 3, 'n_units_l0': 43, 'dropout_l0': 0.4454749279030757, 'n_units_l1': 81, 'dropout_l1': 0.2562585942059985, 'n_units_l2': 99, 'dropout_l2': 0.4183903908296933, 'optimizer': 'SGD', 'lr': 0.005460329367164179}. Best is trial 0 with value: 0.1515625.
[I 2026-01-29 11:20:43,619] Trial 1 finished with value: 0.81328125 and parameters: {'n_layers': 1, 'n_units_l0': 41, 'dropout_l0': 0.413346167836529, 'optimizer': 'Adam', 'lr': 0.001949680512977771}. Best is trial 1 with value: 0.81328125.
[I 2026-01-29 11:20:47,170] Trial 2 finished with value: 0.1078125 and parameters: {'n_layers': 3, 'n_units_l0': 69, 'dropout_l0': 0.4202576066417447, 'n_units_l1': 22, 'dropout_l1': 0.2820404630588114, 'n_units_l2': 26, 'dropout_l2': 0.31773433173792015, 'optimizer': 'Adam', 'lr': 0.05488580694119367

Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  72
  Number of completed trials:  28
Best trials:
  Value:  0.8390625
  Params: 
    n_layers: 1
    n_units_l0: 65
    dropout_l0: 0.3587638756393953
    optimizer: Adam
    lr: 0.009976297477212313
